In [1]:
import pandas as pd
import numpy as np
import os,sys
import re
pwd = os.getcwd()

In [2]:
# reading dataset
kb_df = pd.read_csv(pwd+"//Datasets//Kabita//Input//kabitakitchen.csv", encoding_errors='ignore')
kb_df

,id,commentText,Labels
0,Ugy_CBm-_CKA3YqrzcB4AaABAg,Pudina ptta nhi dalu to,7
1,Ugy9mx9nuTWJu4dRac14AaABAg,Chiken kacha tu ni rhy ga sis,7
2,Ugz8T2MKLYucL3dM9nh4AaABAg,"Hello mam, I love your all recipes.... 😋😋😋\nAl...",4
3,Ugx_1cCjRbCaDgL0FLF4AaABAg,Its awesome recipe plzz make handi chicken in ...,2
4,UgzLhKVAJ6NN3nZXyjN4AaABAg,Yeh jo measurement hai.........kitne logon ke ...,7
...,...,...,...
4895,UgjFXyC0Qhzk5ngCoAEC,i love chole...thank you kabitaji for sharing ...,1
4896,UghP3bitlJuM13gCoAEC,thnakyou mm,1
4897,UghztLZOqvedfXgCoAEC,thanks mam,1
4898,UggX5Fi2Y430zXgCoAEC,u r fabulous,4


In [3]:
kb_df.shape

(4900, 3)

In [4]:
# replacing enter with ';'
kb_df['commentText'] = kb_df['commentText'].str.replace('\n',';')
kb_df

,id,commentText,Labels
0,Ugy_CBm-_CKA3YqrzcB4AaABAg,Pudina ptta nhi dalu to,7
1,Ugy9mx9nuTWJu4dRac14AaABAg,Chiken kacha tu ni rhy ga sis,7
2,Ugz8T2MKLYucL3dM9nh4AaABAg,"Hello mam, I love your all recipes.... 😋😋😋;All...",4
3,Ugx_1cCjRbCaDgL0FLF4AaABAg,Its awesome recipe plzz make handi chicken in ...,2
4,UgzLhKVAJ6NN3nZXyjN4AaABAg,Yeh jo measurement hai.........kitne logon ke ...,7
...,...,...,...
4895,UgjFXyC0Qhzk5ngCoAEC,i love chole...thank you kabitaji for sharing ...,1
4896,UghP3bitlJuM13gCoAEC,thnakyou mm,1
4897,UghztLZOqvedfXgCoAEC,thanks mam,1
4898,UggX5Fi2Y430zXgCoAEC,u r fabulous,4


In [5]:
# removing emojis
import emoji

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

In [6]:
# remove special characters 
import re

def remove_spechar(text):
    r = re.sub(r'\W+', ' ', text)
    return r.strip()

In [7]:
X = list(map(remove_emoji,list(kb_df['commentText'])))
X = list(map(remove_spechar,X))
y=list(kb_df['Labels'])

C:\Users\murth\AppData\Local\Temp/ipykernel_6844/1096523203.py:5: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return emoji.get_emoji_regexp().sub(u'', text)


In [8]:
import torch
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
print(X)

['Pudina ptta nhi dalu to', 'Chiken kacha tu ni rhy ga sis', 'Hello mam I love your all recipes All the ingredients are easily available and your way of explaining is too good', 'Its awesome recipe plzz make handi chicken in handi', 'Yeh jo measurement hai kitne logon ke liye hai', 'Kabita mam I tried ur egg biryani everyone in my house just loved it thank so much that was so delicious it was all because of ur recipe', 'cooker me kar sakte he na', 'Mujhe bhot ache lagi apki respi mene subscribe kardia bhot ache he', 'Mam dahi k jgh p kuch or use kr skte h kya', 'Wooooooo it s very yummmmmm I love it', 'This is a perfect biryani recipe Apko follow kar k banaya acchi bani biryani', 'Hi Didi I was always curious that How Biryani Made Thank you so much for putting this detailed video This Weekend I will try and serve it to family Really Motivatied', 'thanx respect from Madam appne tel nahy dala', 'I made this it taste awesome thank you kabita ji', 'You re amazing', 'nice video', 'Aur kya i

In [10]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True # Whether the model returns all hidden-states.
                                  )


# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()
                                  
def bert_vectorization(text):
    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(text)
    
    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    # Mark each of the tokens as belonging to sentence.
    segments_ids = [1] * len(tokenized_text)
    
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers. 
    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        hidden_states = outputs[2]
    
    # `token_vecs` is a tensor with shape [tokens x 768]
    token_vecs = hidden_states[-1][0]
    
    # Calculate the average of all the token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
    
    return np.array(sentence_embedding)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
bert_vectors = list(map(bert_vectorization,X))

In [12]:
print(bert_vectors)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
bert_df = pd.DataFrame(bert_vectors)
bert_df

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.001131,-0.080694,0.390532,-0.375276,0.342453,0.260397,-0.240222,0.127094,-0.118620,-0.069805,...,-0.149397,-0.379096,-0.112705,-0.117240,0.177878,-0.166222,-0.677480,0.246920,0.077405,0.601715
1,-0.369269,-0.268686,0.580534,-0.382813,0.080795,0.261676,0.247408,-0.462703,-0.174721,-0.329651,...,0.001823,-0.999149,0.102011,0.146176,0.083013,-0.819038,-0.481887,0.277724,0.162703,0.393442
2,0.018679,0.437332,0.197052,0.081329,-0.079719,0.021943,-0.087719,-0.103939,-0.056657,-0.312810,...,-0.065665,-0.652862,-0.202713,-0.267925,0.128328,-0.288147,-0.218526,-0.766253,-0.213581,0.381587
3,-0.187438,-0.211634,0.707054,0.013336,0.450096,-0.227681,-0.076488,-0.369296,0.063925,-0.271845,...,-0.284604,-0.244836,0.177389,-0.274019,0.202539,-0.164929,-0.369508,-0.227268,-0.169633,0.342554
4,-0.175979,0.369497,0.307361,-0.344749,0.315236,0.215114,0.208766,0.118265,0.041888,0.082071,...,-0.277312,-0.656377,-0.388079,0.278152,0.462393,0.023997,-0.599597,-0.169109,0.326911,0.483502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,0.424739,0.442285,-0.087716,0.128557,0.067655,0.007682,-0.396108,0.340329,-0.433762,-0.548567,...,-0.391785,0.043287,-0.216434,0.252146,0.275081,0.306067,-0.353628,-0.335227,-0.198242,0.241418
4896,-0.407385,0.297515,0.655218,-0.245135,0.491840,-0.025152,-0.415222,-0.102503,-0.400478,0.046418,...,-0.048430,0.425603,0.213972,-0.649934,0.296751,-0.375038,-0.417707,0.764755,0.098923,0.724992
4897,-0.000476,0.878712,0.675111,-0.343633,0.443095,-0.442400,-0.114166,1.080091,-0.293992,0.139714,...,-0.785307,0.297543,0.061426,-0.538982,0.556757,-0.185568,-0.610792,0.123501,0.286634,0.131285
4898,-0.205280,0.543250,0.235369,-0.062610,-0.046590,-0.146482,0.227702,0.043045,-0.491875,0.053862,...,0.225746,-0.110642,-0.329536,-0.595502,0.101518,-0.213336,-0.102099,-0.681978,0.344907,0.207164


In [14]:
bert_df.to_csv(pwd+"//Datasets//Kabita//FineTunedTransformers//bert_base_finetuned_vectorized_kabita_dataset.csv",index=False)